The following notebook gives a simple example of the usage of a python class named **fftwrapper**. This class was initially written to abstract away some redundant calculation that are required when calculating FFTs.

NumPy provides a function [numpy.fft.fft()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html#numpy.fft.fft) function which calculates the discrete Fourier Transform (DFT) of a (1D) signal. Many functions in NumPy and SciPy were written to mimic the Matlab API as to reduce the learning curve required; *fft.fft()* included.

As such, the *numpy.fft.fft()* function simply returns the discrete Fourier Transform of the signal, without information about the frequency space. Additionally, the [FFT](https://en.wikipedia.org/wiki/Fast_Fourier_transform) algorithm used to calculate the DFT returns the transform in a format which makes it confusing to visualize, requiring the application of the *numpy.fft.fftshift()* function to place the section of the transform associated with "negative" frequencies to the begining of the data, not at the end.

Also, the actual frenquencies had to be calculated separatly, adding an error prone step to the process (that is, before [numpy.fft.fftfreq()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fftfreq.html#numpy.fft.fftfreq) existed). There is no equivalent to *fftfreq()* in Matlab and one has to do the calculation manually.

The **fftwrapper** module was written (many years ago) to abstract away all these calculations. This allowed to quickly calculate DFT and plot all the information without duplicating code. The module was also ported to Matlab as a previous employer required using Matlab.

This module doesn't do much, but reduced significantly code duplication and bugs in a quickly evolving environment. I knew I could trust the DFT, including the calculated spectrum and phases.

Let's import NumPy, matplotlib and plotly:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py

We'll also need the custom FFT wrapper. It's a simple python class in the current directory:

In [2]:
import fftwrapper

Let's define the frequencies variables:

In [3]:
f0s = np.array([1.0, 5.0, 2.5]) # Center frequencies
T0s = 1.0 / f0s                 # Periods
o0s  = 2.0 * np.pi * f0s        # Angular frequencies

The duration of the whole signal will be 10 times the longest period:

In [4]:
nT = 10.0

Other time variables:

In [5]:
nt = 200                        # Nb of time steps
tmax = nT * T0s.max()
time = np.linspace(0.0, tmax, nt)
dt = time[1] - time[0]

Let's build the total signal:

In [6]:
signal = np.zeros(nt, dtype=np.complex128)
for i in range(len(T0s)):
    # Real signal: negative and positive frequencies
    #signal += np.cos(o0s[i] * time)
    # Complex signal: only positive frequencies
    signal += np.cos(o0s[i] * time) + 1.0j*np.sin(o0s[i] * time)

Calculate the FFT using the wrapper:

In [7]:
fft = fftwrapper.FFT()
fft.Set_Time_Signal(time, signal, resize_NFFT = 0)        # No resizing
#fft.Set_Time_Signal(time, signal, resize_NFFT = -1)       # Resize to lower closest power of 2
#fft.Set_Time_Signal(time, signal, resize_NFFT = +1)       # Resize to upper closest power of 2
#fft.Set_Time_Signal(time, signal, resize_NFFT = 10*nt)    # Resize to specific value

We are ready to plot!

In [8]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(223)
ax3 = fig.add_subplot(224, sharex=ax2)

ax1.plot(fft.t, fft.signal.real, lw=2, color='b')
ax1.plot(fft.t, fft.signal.imag, lw=2, color='r')
ax1.set_xlabel("t [time]")
ax1.set_ylabel("Signal [arb. unit]")

# Plot |FFT|
ax2.plot(fft.f, fft.Sabs, '-', lw=2, color='m')
ax2.set_xlabel(r"frequencies [time$^{-1}$]")
ax2.set_ylabel(r"Spectrum")
ax2.set_yscale('log')

# Plot /_ FFT
ax3.plot(fft.f, fft.phase, '-', lw=2, color='g')
ax3.set_xlabel(r"frequencies [time$^{-1}$]")
ax3.set_ylabel(r"Phase")

We could plot the figure here:

In [9]:
#plt.show()

But instead, we'll convert the matplotlib figure to plotly:

In [10]:
py.iplot_mpl(fig)